In [ ]:
import random
from langchain import PromptTemplate, FewShotPromptTemplate

In [ ]:
from datasets import load_dataset

dataset = load_dataset("wmt19","de-en")

In [ ]:
import random
sid = random.sample(range(len(dataset["train"])), k=5)

In [ ]:
dataset["train"][0]

In [ ]:
pair = "de-en"
src_lang, tgt_lang = pair.split("-")

In [ ]:
# examples = [
#     {
#         "src_lang": "en",
#         "src": "Hello",
#         "tgt_lang": "fr",
#         "ref": "Bonjour"
#     },
#     {
#         "src_lang": "en",
#         "src": "Goodbye",
#         "tgt_lang": "fr",
#         "ref": "Au revoir"
#     }
# ]
examples = [
    {
        "src_lang": src_lang,
        "src": sample[src_lang],
        "tgt_lang": tgt_lang,
        "ref": sample[tgt_lang]
    }
    for sample in random.sample(dataset["train"], k=5)
]
example_prompt = PromptTemplate(
    template="{src_lang}: {src}\n{tgt_lang}:{ref}",
    input_variables = ["src_lang", "src", "tgt_lang", "ref"],
)
few_shot_prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    # prefix="Give the antonym of every input",

    suffix="{src_lang}: {src}\n{tgt_lang}:",
    input_variables=["src_lang", "src", "tgt_lang"],
    example_separator="\n",
)
print(few_shot_prompt.format(
    src_lang="en",
    src="Hello",
    tgt_lang="fr",
))

In [ ]:
import pandas as pd

df = pd.read_csv("./data/wmt19_en-de.csv")

In [ ]:
for i, row in df.iterrows():
    prompt = row["input"]
    # encoded_prompt = tokenizer.encode(prompt, bos=True, eos=False, device=fabric.device)
    # y = generate(
    #         model,
    #         encoded_prompt,
    #         max_new_tokens,
    #         model.config.block_size,  # type: ignore[union-attr,arg-type]
    #         temperature=temperature,
    #         top_k=top_k,
    #     )
    y = "kek"
    df.loc[i, "mt"] = y
df.to_csv("./data/wmt19_en-de_translated.csv", index=False)

    

In [ ]:
df